In [1]:
%run "Source Code/Setup Notebook.ipynb"

<img class="img-responsive" width="35%" src="http://www.cytoscape.org/images/cytoscape3_logo.png" alt="logo">

# Welcome to Cytoscape!

* <font color="orange">**Cytoscape**</font> is a [jupyter](http://jupyter.org/index.html) based framework to view graphical networks, ** *no coding knowledge required* **.
* The entire platform is cloud based so you don't need to download anything, just login and start.
* Furthermore, sharing scientific results is a breeze. When you're done with this notebook, give it to your colleagues and they can instantly replicate it on <font color="orange">Cytoscape</font>.

<font color="red">**Before you continue**</font> please watch the tutorial video below to get a better idea of what the <font color="orange">Cytoscape</font> software is and how it works.

## Video Proposal <small>**(July 19, 2018)**</small>

In [7]:
HTML("""
<div class="row"><div class="col-xs-12 col-md-offset-3 col-md-6">
<div class="embed-responsive embed-responsive-16by9">
  <iframe class="embed-responsive-item" 
  src='https://www.youtube.com/embed/p34OR7IVZiE'></iframe></div>
</div></div>
""")

## <font color="red">Old</font> Video Proposal <small>**(Feb 20, 2018)**</small>

In [8]:
HTML("""
<div class="row"><div class="col-xs-12 col-md-offset-3 col-md-6">
<div class="embed-responsive embed-responsive-16by9">
  <iframe class="embed-responsive-item" 
  src='https://www.youtube.com/embed/cKzaGvPu7-8'></iframe></div>
</div></div>
""")

# Create A  Network

In this section will will choose a dataset to work on. There are **2** ways to create a datast:

1. Choose a dataset on this machine from [my_data](./my_data)
2. Choose a dataset from **NDEx**

<font color="magenta">**From my_data : **</font>The folder, [my_data](./my_data), contains datasets that you can load into this notebook. You can also upload a dataset from your computer to the [my_data](./my_data) folder. To view new files added to [my_data](./my_data) update the program by running all the cells again.


<font color="magenta">** NDEx : **</font> A dataset can be downloaded from the NDEx as well. Type your <font color="green">username</font>, <font color="green">password</font>, and the <font color="green">URL</font> of the NDEx network to download it.

In [9]:
%run "Source Code/Create Network.ipynb"

Button(button_style='info', description='Create', style=ButtonStyle(), tooltip='Use the above box to import da…

# Visualize 

#  <font color="lightBlue">Old Visualize</font>

## Data Summary

Now that you have chosen a dataset, lets look at some basic data features to get a better understanding of what is happening. **Click the button** below to see how many <font color="blue">nodes</font> and  <font color="blue">edges</font> your data contains, and to view the first few datapoints.

In [11]:
# We now get the data's filename according to the user's selection from the tab widget
summary_btn= ipw.Button(description='View Summary',button_style='info', tooltip="Click for Data Summary" )
display(summary_btn)

def on_summary_clicked(b):
    clear_output(); display(summary_btn)  #<-- in case the button was clicked before, we reset output
    if type(df) == type(None):   #<-- this occures if user didn't click the "Get Data" button 
        display(HTML("<h3 style='color: red'>You must First Build a Dataset</h3>"))
    else :
        display(HTML("<hr><p><b>Number of Nodes : </b>"+str(len(nodes))+"</p> \
                      <p><b>Number of Edges : </b>"+str(len(df))+"</p>"))
        display(df[0:7])
summary_btn.on_click(on_summary_clicked)

A Jupyter Widget

## Create Graph

We will now visualize our dataset using a graph. You can use the tools below to change the <font color="blue">layout</font> of where nodes are placed as well as to change the graph's <font color="blue">color</font>. The  <font color="blue">layout</font> tool is especially useful for examining different characteristics in your data. After you decide the graph's settings, click the button below to **create your graph**.

In [12]:
# We save our graph into an html file and call the html file to create the graph
# This is the basic template for the html, we then add the javascript code of the graph to it
html = """<html>
    <head><script src="http://blog.js.cytoscape.org/public/demos/getting-started/cytoscape.js"></script></head>
    <style> #cy { width: 100%; height: 100%; position: absolute; top: 0px; left: 0px;} </style>
    <body><div id="cy"></div></body>
</html>\n\n"""

In [13]:
# Helper Functions: getLayout and getStyle

def getLayout(layout): #<-- creates the layout when we create the graph
    return "layout: { name: '"+layout+"' }, \n"

def getStyle( color ): #<-- uses widg_color to create the style (specifically the color) of the graph
    return "style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': '"+color+"' } },\n \
{ selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': '"+color+"'} } ], \n "

In [14]:
# We create the widgets for graph settings
widg_layout = ipw.Dropdown(options=['cose','circle','grid','random','breadthfirst'], description='Layout : ')
widg_color = ipw.ColorPicker(  description='Color : ', value='gray' )
display( ipw.HBox([widg_layout ,widg_color]) )

A Jupyter Widget

In [15]:
js_head = "<script> var cy = cytoscape({ container: document.getElementById('cy'), \n" #<-- starting code
js_tail="\n }); </script>" #<-- ending code

In [16]:
### Helper Function: getElements(df)
# This create the nodes/edges in the correct format for cytoscape.js
def getElements(df):
    if data_local:
        # Add nodes to the js file
        js_elements = "elements: [ \n"
        for i in nodes:
            js_elements += "{ data: { id: '"+i+"'} }, \n"

        # Add edges to the js file
        for i in range(len(df)):
            word = df.iloc[i].values
            js_elements += "{data: { id:'"+str(i)+"', source:'"+ str(word[0])+"', target:'"+str(word[2])+"' } }, \n"    
        js_elements += "]\n"
    
    else:
        # Add nodes to the js file
        js_elements = "elements: [ \n"
        for key,node in cx.get_nodes() :
            js_elements += "{ data: { id: '"+str(key)+"'} }, \n"

        # Add edges to the js file
        for key,edge in cx.get_edges() :
            js_elements += "{data: { id:'"+str(key)+"', source:'"+ str(edge.get_source())+"', target:'"+str(edge.get_target())+"' } }, \n"    
        js_elements += "]\n"
    
    return js_elements

In [17]:
# We now get the data's filename according to the user's selection from the tab widget
graph_btn= ipw.Button(description='Create Graph!',button_style='info', tooltip="Click to Create Graph!" )
display(graph_btn)

def on_graph_clicked(b):
    clear_output(); display(graph_btn)  #<-- in case the button was clicked before, we reset output
    
    df = False
    if type(df) == type(None):   #<-- this occures if user didn't click the "Get Data" button 
        display(HTML("<h3 style='color: red'>You must First Build a Dataset</h3>"))
    # This creates the graph!
    else : 
        # Get js templates with corresponding helper functions
        js_layout = getLayout( widg_layout.value )
        js_style  = getStyle( widg_color.value )
        js_elements = getElements(df)
           
        js = js_head+ js_layout+ js_style+ js_elements+ js_tail #<-- create final js file of graph
        with open("./source_code/graph.html","w") as file: #<-- Add html and js to the HTML file
            file.write( html+js )
        display(HTML("""<hr>
        <div class="embed-responsive embed-responsive-4by3">
          <iframe class="embed-responsive-item" 
          src='./source_code/graph.html'></iframe>
        </div><hr>
        """)) #<-- display the newly created html file

graph_btn.on_click(on_graph_clicked)

A Jupyter Widget

# Save / Export

In [3]:
wUpload_username = ipw.Text(description='Username')
wUpload_password = ipw.Password(description='Password :')
wUpload_name = ipw.Text(description='Model Name')
wUpload_ndex = ipw.HBox([ wUpload_username, wUpload_password  ])
wUpload = ipw.VBox( [wUpload_ndex, wUpload_name] )
wUpload

In [19]:
upload_btn= ipw.Button(description='Upload',button_style='info', tooltip="Fill in the info, then click to upload" )
def upload_model(b):
    account = wUpload_username.value
    password = wUpload_password.value
    name = wUpload_name.value
    url="http://www.ndexbio.org"
    
    cx.set_name( name )
    #cx.add_network_attribute(name='description', values= description  ) 
    
    try:
        my_ndex= ndex2.client.Ndex2( url, account, password)
        my_ndex.update_status()
        #print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (account, password))
        print(inst.args)
    
    upload_id   = cx.upload_to( url, account, password).split("network/")[1] #<-- "cx"  is a global variable
    upload_text = "http://www.ndexbio.org/#/network/"+upload_id 
    
    display(HTML("<hr><font size=4><b style='color: magenta'>Uploaded Network at : </b> <a href='"+widg2_url.value+"'>This Location</a>  ""</font>"))
    
upload_btn.on_click( upload_model )
upload_btn

A Jupyter Widget

In [17]:
# This just adds a little space to the bottom of the notebook
display(HTML("<p>&nbsp</p><p>&nbsp</p><p>&nbsp</p><p>&nbsp</p><p>&nbsp</p>"))